## Problem Statement
- **Role**: Strategist at a leading e-commerce company.
- **Task**: Prepare for the upcoming festive season by analyzing competitors' discounts and promotional offers.
- **Objective**: Design a competitive strategy to either match or surpass the competition's offers.

## What You Will Build
An **Agentic System** that can:
- **Dynamically code** to handle changing website structures.
- **Scrape the web** to extract discounts and promotional offers.
- **Summarize data** into actionable insights for decision-making.

## Learning Outcomes
By the end of this notebook, you should be able to:
- Configure AutoGen agents with and without LLMs.
- Run local code execution safely in a controlled directory.
- Orchestrate a multi-agent workflow (writer → reviewer → executor).

## Outcome
Enable data-driven decision-making to maintain a competitive edge during the festive season.



In [ ]:
import autogen
import os
from IPython import get_ipython
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor
from IPython.display import HTML, display

# Core imports:
# - autogen: multi-agent orchestration framework
# - LocalCommandLineCodeExecutor: runs code safely in a local working folder
# - IPython display utilities: useful for rich notebook output

In [ ]:
from dotenv import load_dotenv

# Load environment variables (e.g., API keys) from a .env file.
# If your .env lives elsewhere, pass an explicit path to load_dotenv("/path/to/.env").
load_dotenv()

True

# Local Code Execution
In this section, we create a local executor and connect it to an agent so that any generated code runs in a controlled folder with a time limit.

In [12]:
user_proxy = UserProxyAgent(
    name="User",
    # Disable LLM use for this agent (acts as a controller/user).
    llm_config=False,
    # Stop the conversation when a message contains the word TERMINATE.
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    # Only request human input when termination is reached.
    human_input_mode="TERMINATE",
    # This agent does not execute code directly.
    code_execution_config=False
)

In [13]:
code_writer_agent = ConversableAgent(
    name="CodeWriter",
    # System prompt guides the agent's behavior and stop condition.
    system_message="""You are a Python developer.
    You use your coding skill to solve problems.
    Once the task is done, returns 'TERMINATE'.""",
    # Use a lightweight model for fast code generation.
    llm_config={"config_list": [{"model": "gpt-4o-mini"}]},
)

In [14]:
code_reviewer_agent = ConversableAgent(
    name="CodeReviewer",
    # Reviewer checks the generated code against requirements.
    system_message="""You are an experienced software developer
    Review the code given to you based on the requirements
    Once the task is done, returns 'TERMINATE'.""",
    # Same model as writer for consistency.
    llm_config={"config_list": [{"model": "gpt-4o-mini"}]},
)

In [15]:
local_executor = LocalCommandLineCodeExecutor(
    # Safety: limit execution time (seconds) for any generated script.
    timeout=20,
    # All generated files will be created inside this working directory.
    work_dir='./code files')

In [16]:
local_executor_agent = ConversableAgent(
    "local_executor_agent",
    # No LLM here; this agent only runs code on the local machine.
    llm_config=False, 
    # Attach the local executor for running generated scripts.
    code_execution_config={"executor": local_executor},
    # ALWAYS means the notebook may prompt for confirmation.
    human_input_mode="ALWAYS",
)

In [17]:
# Two-step instruction set:
# 1) ask the writer to generate code
# 2) ask the reviewer to check it before execution
messages = ["""To check whether there are any offers or discounts available on a given e-commerce website -
                https://www.flipkart.com/
                Follow these steps,
                1. download the html page of the given URL
                2. we only need text content, so remove any CSS, JavaScript, and Image content
                3. save the remaining html content.
               """ ,
    """The code is given by an LLM. Review the code based on the above requirements and provide it for execution"""]

In [19]:
# Run two sequential chats:
# - User → CodeWriter (generate code)
# - Local executor agent → CodeReviewer (review before execution)
chat_results = autogen.initiate_chats(
    [
        {
            "sender": user_proxy,
            "recipient": code_writer_agent,
            "message": messages[0],
            "summary_method": "last_msg",
        },
        {
            "sender": local_executor_agent,
            "recipient": code_reviewer_agent,
            "message": messages[0] + messages[1],
            "summary_method": "last_msg",
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to CodeWriter):

To check whether there are any offers or discounts available on a given e-commerce website -
                https://www.flipkart.com/
                Follow these steps,
                1. download the html page of the given URL
                2. we only need text content, so remove any CSS, JavaScript, and Image content
                3. save the remaining html content.
               

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
CodeWriter (to User):

To achieve the specified task, we can use Python with the `requests` library to download the HTML content and the `BeautifulSoup` library from `bs4` to parse the HTML and remove unwanted elements such as CSS, JavaScript, and images.

Here's a step-by-step outli

In [20]:
# Inspect the artifacts generated by the local executor.
os.listdir('./code files/')

['tmp_code_bf2ed78c30d35d8dee08d27be3c7ec79.py',
 'cleaned_flipkart_content.html']

### Weekly Workout Schedule

#### Monday
- **Activity:** Full Body Strength Training
- **Duration:** 45 minutes
- **Focus:** Compound movements (squats, deadlifts, bench press, rows)

#### Tuesday
- **Activity:** Cardio (Running or Cycling)
- **Duration:** 30 minutes
- **Intensity:** Moderate pace

#### Wednesday
- **Activity:** Rest Day
- **Focus:** Recovery and light stretching

#### Thursday
- **Activity:** Upper Body Strength Training
- **Duration:** 45 minutes
- **Focus:** Push-ups, pull-ups, shoulder press, bicep curls, tricep dips

#### Friday
- **Activity:** Cardio (HIIT Workout)
- **Duration:** 30 minutes
- **Intensity:** High-intensity intervals (30 seconds work, 30 seconds rest)

#### Saturday
- **Activity:** Lower Body Strength Training
- **Duration:** 45 minutes
- **Focus:** Lunges, leg press, calf raises, glute bridges

#### Sunday
- **Activity:** Rest Day
- **Focus:** Recovery, yoga, or light walking

### Notes:
- Ensure to warm up before and cool down after each workout.
- Stay hydrated and listen to your body; adjust intensity as needed.
- Aim for at least 150 minutes of moderate aerobic activity throughout the week.

.